In [2]:
import json
from pathlib import Path
import pandas as pd
import geopandas as gpd

In [3]:
ROOT = Path('../..')

OUTPUT_DIR = ROOT / 'src/_data/areas/'

REF_DIR = OUTPUT_DIR / 'reference'
REF_DIR.mkdir(parents=True, exist_ok=True)

MAPS_DIR = OUTPUT_DIR / 'maps'
MAPS_DIR.mkdir(parents=True, exist_ok=True)

DATA_DIR = ROOT / 'src/generated/areas/constituency/_data/'
DATA_DIR.mkdir(parents=True, exist_ok=True)

In [4]:
geo = gpd.read_file(ROOT / "working/upstream/uk-constituencies-2024.geojson").query('PCON24CD.str.startswith("E")')
geo.crs = None

In [53]:
mapping = pd.read_json(ROOT / 'working/upstream/uk-constituencies-2024-changes.json', orient="index")

In [54]:
geo.loc[:, ['PCON24CD']].set_index('PCON24CD').merge(mapping, left_index=True, right_index=True).rename_axis('PCON24CD').reset_index().to_json(REF_DIR / 'pcon.json', orient="records")

In [55]:
geo.to_file(MAPS_DIR / 'pcon.geojson', coordinate_precision=5)

In [57]:
with open(MAPS_DIR / 'pcon.geojson') as geojson:
    d = json.load(geojson) 

In [58]:
res = {
    f['properties']['PCON24CD']: {
        "type": "FeatureCollection",
        "features": [f]
    } for f in d['features']
}

with open(DATA_DIR / 'map.json', 'w') as f:
    json.dump(res, f)